# Building Out Functions

In [83]:
# importing libraries
import pandas as pd # data manipulation

from tika import parser # pdf bill text extraction
from wwo_hist import retrieve_hist_data # weather data api
import re # text formatting with regex

import arrow # date formatting
import datetime as dt # date formatting

import env # encoded data

In [2]:
# storing august and september bill filepaths (september is for testing)
filePath_aug = 'pdfs/2022-08-25_bill.pdf'
filePath_sep = 'pdfs/2022-09-27_bill.pdf'

In [3]:
# creating function for extracting text 
def extract_pdf(filePath):
    '''
    Argument(s)
        - filePath (str): path to the pdf file to be extracted
    Returns 
        - text (str): extracted pdf text
    '''
    
    text = parser.from_file(filePath)['content']
    print(f'Text extraction from {filePath} complete.')
    
    return text

In [ ]:
# text_aug = extract_pdf(filePath_aug)
# text_aug[:1000]

In [33]:
# working out code to loop through each month's pdf and store text in variable
for file in [filePath_aug, filePath_sep]:
    print(file[7:15])
    print(arrow.get(file[10:12], 'MM').format('MMMM'))
    print(arrow.get(file[7:15], 'YY-MM-DD').format('MMM DD, YYYY'))
    print(arrow.get(file[7:15], 'YY-MM-DD').format('YYYY-MM-DD'))
    print()

22-08-25
August
Aug 25, 2022
22-08-25

22-09-27
September
Sep 27, 2022
22-09-27



# 🗑️ Didn't work 
Loop didn't store variables but saving code to reuse code parts that did work

In [ ]:
# # testing loop
# for file in [filePath_aug, filePath_sep]:
#     print(arrow.get(file[10:12], 'MM').format('MMM').lower())
#     exec(f'text_{arrow.get(file[10:12], "MM").format("MMM").lower()} = extract_pdf(file)')

In [ ]:
# def loop_extract_pdfs(filePath_list):
#     '''
#     Argument(s)
#         - filePath_list (list): path to the pdf file to be extracted
#     Stores
#         - extracted PDF text as str, variable name format: text_MMM (e.g. text_aug)
    
#     This function takes a list of pdf file paths, loops through them, applying the extract text function,
#     storing the extracted text based on the pdf month. 
#     '''    
    
#     for file in filePath_list:
#         exec(f'text_{arrow.get(file[10:12], "MM").format("MMM").lower()} = extract_pdf(file)')
#         print(f"Text extracted for {arrow.get(file[7:15], 'YY-MM-DD').format('MMM DD, YYYY')} billing date.")
#         print(f'****Variable Name*****\ntext_{arrow.get(file[10:12], "MM").format("MMM").lower()}')
#         print()
        

In [ ]:
# loop_extract_pdfs([filePath_aug, filePath_sep])

In [ ]:
# # verifying extracted text stored
# text_aug

In [ ]:
# text_sep

In [ ]:
# # storing file paths variables for each month in list
# filePath_list = [filePath_aug, filePath_sep]

In [ ]:
# # initiating empty dictionary to store extracted text as values and month labels as keys
# text_dict = {}

# # initializing empty list to store dictionary keys
# text_list = []

# # looping through each file path to extract text
# for file in filePath_list:
#     text_dict[f'text_{arrow.get(file[10:12], "MM").format("MMM").lower()}'] = extract_pdf(file)
#     text_list.append(f'text_{arrow.get(file[10:12], "MM").format("MMM").lower()}')
#     print(f'Dictionary Key: text_{arrow.get(file[10:12], "MM").format("MMM").lower()}')
#     print()
    
# # text_dict

In [ ]:
# # verifying can access values in dictionary
# text_list

In [ ]:
# # verifying can access values through looping text_list
# for value in text_dict.values():
#     print(value)

> Creating function that loops through list of pdf file paths and stores the text of those files

In [68]:
def looped_FileExtraction(filePath_list):
    '''
    This function takes in a list of pdf file paths, loops through that list and stores the extracted
    text as dictionary values, creating associated key names based on the pdf name (bill month), and 
    reading that dictionary to pandas df.
    
    Args:
        - filePath_list (list): list of pdf file paths as strings
    
    Returns:
        - df: 
    '''
    # initiating empty dictionary to store extracted text as values and month labels as keys
    text_dict = {}

    # initializing empty list to store dictionary keys
    text_list = []

    # looping through each file path to extract text using extract_pdf() function
    for file in filePath_list:
        text_dict[f'{arrow.get(file[7:15], "YY-MM-DD".format("YYY-MM-DD"))}'[:10]] = extract_pdf(file)
        text_list.append(f'text_{arrow.get(file[7:15], "YY-MM-DD".format("YYY-MM-DD"))}')
        
    # writing dictionary to df
    df = pd.DataFrame.from_dict(text_dict, orient='index', columns = ['text'])
    df.reset_index(inplace = True)
    df.rename(columns = {'index':'bill_date'}, inplace = True)
    
    return df

In [70]:
# verifying function works
df = looped_FileExtraction([filePath_aug, filePath_sep])
df

Text extraction from pdfs/2022-08-25_bill.pdf complete.
Text extraction from pdfs/2022-09-27_bill.pdf complete.


,bill_date,text
0,2022-08-25,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,2022-09-27,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


## Text Extraction Function

In [92]:
def extract_text(text, str_start, str_end):
    '''
    This functiont takes in a full text string and returns the text extracted from between the string start
    index and string end index. 
    '''
    start_index = text.index(str_start) + len(str_start)
#     print(f'Starting Index: {start_index}')
    end_index = text.index(str_end, start_index)
#     print(f'Ending Index: {end_index}')
    
    extracted_text = text[start_index:end_index]
    
    return extracted_text

In [93]:
# testing function
extract_text(df.text[0], f'{env.zip}\n\n', '\n\nOn or Before')

'08/25/22'

Data to be parsed
### Billing Info
✅ Bill Detail | Label | Description
:-- | :-- | :--
✅ Billing Date | `bill_date` | YYYY-MM-DD
✅ Billing Period Start | `bill_start` | YYYY-MM-DD | Start date of the billing period
✅ Billing Period End | `bill_end` | YYYY-MM-DD | End date of the billing period
✅ Billing Period (days) | `bill_period_days` | *int*, number of days in billing period, `bill_end` - `bill_start`
✅ Meter Read | `meter` | The meter reading

In [71]:
# how to get series of each bill's extracted text
df.text

0    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
Name: text, dtype: object

In [108]:
# # how to isolate each row text
# df.text[0]

In [153]:
# working on loop that will be used in function to extract the rest of the billing date data
for text in df.text:
    # extacting raw billing period
    test_dateData = extract_text(text, 'Billing Period ', '\nYour next')
    print(f'Parsed Bill Period: {test_dateData}')
    
    # extracting bill start
#     bill_start = ''.join(re.findall(r'^[a-zA-Z0-9\s]+[\s\s]+[0-9?\s,0-9]+', test_dateData))
    bill_start = test_dateData.split('-')[0].strip()
    # formatting date
    bill_start = arrow.get(bill_start, 'MMM DD, YYYY').format('YYYY-MM-DD')
    print(f'Parsed Bill Start: {bill_start}')
    
    # extracting bill end
#     bill_end = ''.join(re.findall(r'-[a-zA-Z0-9\s]+[\s\s]+[0-9?\s,0-9]+', test_dateData))
    bill_end = test_dateData.split('-')[1].strip()
    # formatting date
    bill_end = arrow.get(bill_end, 'MMM DD, YYYY').format('YYYY-MM-DD')
    print(f'Parsed Bill End: {bill_end}')
    
    # using bill start and end dates to get number of days in billing period
    bill_days = (pd.to_datetime(bill_end) - pd.to_datetime(bill_start)).days + 1
    print(f'Parsed Bill Period Duration: {bill_days} days')
    
    print()

Parsed Bill Period: Jul 26, 2022 - Aug 23, 2022
Parsed Bill Start: 2022-07-26
Parsed Bill End: 2022-08-23
Parsed Bill Period Duration: 29 days

Parsed Bill Period: Aug 24, 2022 - Sep 23, 2022
Parsed Bill Start: 2022-08-24
Parsed Bill End: 2022-09-23
Parsed Bill Period Duration: 31 days



In [138]:
extract_text(text, 'Billing Period ', '\nYour next').split('-')

['Jul 26, 2022 ', ' Aug 23, 2022']

In [164]:
# for loop to extract meter reading, read date, and consumption
for text in df.text:
    
    # extacting raw billing period
    test_dateData = extract_text(text, 'Billing Period ', '\nYour next')
#     print(f'Parsed Bill Period: {test_dateData}')
    
    # extracting bill start
#     bill_start = ''.join(re.findall(r'^[a-zA-Z0-9\s]+[\s\s]+[0-9?\s,0-9]+', test_dateData))
    bill_start = test_dateData.split('-')[0].strip()
    # formatting date
    bill_start = arrow.get(bill_start, 'MMM DD, YYYY').format('YYYY-MM-DD')
    print(f'Parsed Bill Start: \t\t{bill_start}')
    
    # extracting bill end
#     bill_end = ''.join(re.findall(r'-[a-zA-Z0-9\s]+[\s\s]+[0-9?\s,0-9]+', test_dateData))
    bill_end = test_dateData.split('-')[1].strip()
    # formatting date
    bill_end = arrow.get(bill_end, 'MMM DD, YYYY').format('YYYY-MM-DD')
    print(f'Parsed Bill End: \t\t{bill_end}')
    
    # using bill start and end dates to get number of days in billing period
    bill_days = (pd.to_datetime(bill_end) - pd.to_datetime(bill_start)).days + 1
    print(f'Parsed Bill Period Duration: \t{bill_days} days')
    
    #################
    
    list_ = extract_text(text, 'R-', '#6271330').replace(')', '').replace(',', '').split()
#     print(f'List values: {list_}')
    
    meter_date = arrow.get(list_[0], 'MM/DD/YYYY').format('YYYY-MM-DD')
    print(f'Meter read date: \t\t{meter_date}')
    
    meter = int(list_[1][:5])
    print(f'Current meter reading: \t\t{meter}')
    
    consumption = int(list_[2])
    print(f'Consumption: \t\t\t{consumption}')
    
    peak_all = int(consumption - 600)
    print(f'Peak consumption for all 12mos: {peak_all}')
    
    print()

Parsed Bill Start: 		2022-07-26
Parsed Bill End: 		2022-08-23
Parsed Bill Period Duration: 	29 days
Meter read date: 		2022-08-23
Current meter reading: 		78953
Consumption: 			1164
Peak consumption for all 12mos: 564

Parsed Bill Start: 		2022-08-24
Parsed Bill End: 		2022-09-23
Parsed Bill Period Duration: 	31 days
Meter read date: 		2022-09-23
Current meter reading: 		79963
Consumption: 			1010
Peak consumption for all 12mos: 410



### Charges
Charge Detail | Label | Description
:-- | :-- | :--
*Service Availablity Charge | `service_charge` | *float*, Covers the cost of metering and billing for your address, regardless of consumption
✅ Energy Consumption | `consumption` | kWh Used
Base rate | `consumption_rate` | *float*, per kWh charge for consumption
Energy Charge | `consumption_charge` | *float*, Recovers costs for power plants and other infrastructure based on the amount of electricity you use, `consumption` * `consumption_rate`
*Peak Consumption | `peak` | *float*, kWh used in excess of 600
✅ Peak Consumption - All Months | `peak_all`| *float*, kWh used in excess of 600 for all months 
Peak Consumption Rate | `peak_rate` | *float*, per kWh charge for peak consumption
Peak Consumption Charge | `peak_charge` | *float*, The Peak Capacity Charge covers the higher costs for electricity CPS Energy incurs during summer months when demand for electricity is at its highest
*Fuel Adjustment Rate | `fuel_rate` | *float*, The cost for fuel above the base rate. 
Fuel Adjustment Charge | `fuel_charge` | *float*, `fuel_rate` * `consumption`, The fuel adjustment covers the difference between the base rate customers are charged for natural gas use and how much it actually costs to get and deliver that gas. This charge fluctuates with the market price of natural gas.
Regualtory Adjustment Rate | `regulatory_rate` | *float*, This covers two costs — the cost of statewide transmission and the cost CPS Energy pays to be a part of the Electric Reliability Council of Texas (ERCOT).
Regulatory Adjustment Charge | `regulatory_charge` | *float*, `regulatory_rate` * `consumption` 
Total Bill Amount | `total_bill` | *float*, (1) `service_charge` + (2) `consumption_charge` + (3) `peak_charge` + (4) `fuel_charge` + (5) `regulatory_charge`

### Weather Data
Weather Detail | Label | Description
:-- | :-- | :--
Weather Date | `date` | Date of weather observation
Feels Like Temp | `temp_feels`
Max Temp | `temp_max`| 
Min Temp | `temp_min`| 
Temp Spread | `temp_spread`| `temp_max` = `temp_min`
Humidity | `humidity`| 
Heat Index | `heat_index`| 
Wind Chill | `wind_chill`| 
Wind Gust | `wind_gust`| 
Cloud Cover | `cloud_cover`| 
Precipitation | `precip`| 
Sun Hours | `sun_hrs`| 
Sunrise | `sunrise`| 
Sunset | `sunset`| 